In [1]:
%load_ext autoreload
%autoreload 2

from buffer import AllActivationBuffer
from trainers.top_k import TrainerSCAE, AutoEncoderTopK
from training import trainSCAE

from datasets import load_dataset
import torch as t
from nnsight import LanguageModel
from tqdm import tqdm


device = "cuda:0" if t.cuda.is_available() else "cpu"
model = LanguageModel("gpt2", device_map=device, torch_dtype=t.bfloat16)
# model.eval()


dataset = load_dataset(
    'Skylion007/openwebtext', 
    split='train', 
    streaming=True,
    trust_remote_code=True
    )

class CustomData():
    def __init__(self, dataset):
        self.data = iter(dataset)

    def __iter__(self):
        return self

    def __next__(self):
        return next(self.data)['text']

data = CustomData(dataset)

In [ ]:
C = 10
expansion = 16
k = 64 # TODO: automatically detect these

num_features = model.config.n_embd * expansion

In [3]:
n_layer = model.config.n_layer

submodules = {}
for layer in range(n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")


buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    d_submodule=model.config.n_embd, # output dimension of the model component
    n_ctxs=128,  # you can set this higher or lower depending on your available memory
    device="cuda",
    out_batch_size = 16,
    refresh_batch_size = 256,
    dtype=t.bfloat16,
) 

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
def random_up_features(down_layer):
    # Fake important-connection diciontary, for testing
    dic = {}
    for layer in range(down_layer):
        dic[f"mlp_{layer}"]  = t.randint(0, num_features, (num_features, C), dtype=t.long)
        dic[f"attn_{layer}"] = t.randint(0, num_features, (num_features, C), dtype=t.long)
    return dic

important_features = {f"mlp_{down_layer}": random_up_features(down_layer) 
                      for down_layer in range(n_layer)}
                        

# Get submodule names from the submodules dictionary
submodule_names = list(submodules.keys())

pretrained_info = {}
for layer in range(model.config.n_layer):
    for module in ['attn', 'mlp']:
        pretrained_info[f'{module}_{layer}'] = {'repo_id': 'jacobcd52/scae', 'filename': f'ae_{module}_{layer}.pt'}

In [ ]:
trainer = TrainerSCAE(
        activation_dims={name: model.config.n_embd for name in submodule_names},
        dict_sizes={name: model.config.n_embd * expansion for name in submodule_names},
        ks={name: k for name in submodule_names},
        submodules=submodules,
        important_features=important_features,
        pretrained_info=pretrained_info,
        model_config=model.config,
        auxk_alpha=0,
        connection_sparsity_coeff=0.1,
        use_sparse_connections=True,
        seed=None,
        device="cuda",
        wandb_name="SCAE",
        dtype=t.bfloat16, 
)

In [ ]:
input_acts, target_acts = next(buffer)
trainer.update(0, input_acts, target_acts)

In [ ]:
def run_evaluation(trainer, buffer, n_batches=100, ce_batch_size=32):
    varexp_metrics = {name : {} for name in trainer.submodules.keys()}
    ce_metrics = {name : {} for name in trainer.submodules.keys()}
    for i in tqdm(range(n_batches)):
        # get varexp metrics
        input_acts, output_acts = next(buffer)
        batch_varexp_metrics = trainer.evaluate_varexp_batch(input_acts, output_acts, use_sparse_connections=False)

        # get CE metrics
        b = buffer.refresh_batch_size
        buffer.refresh_batch_size = ce_batch_size
        tokens = buffer.token_batch()
        batch_ce_metrics = trainer.evaluate_ce_batch(model, tokens, use_sparse_connections=False)
        buffer.refresh_batch_size = b

        for name in ce_metrics.keys():
            for metric in batch_ce_metrics[name].keys():
                ce_metrics[name][metric] = ce_metrics[name].get(metric, 0) + batch_ce_metrics[name][metric]
            for metric in batch_varexp_metrics[name].keys():
                varexp_metrics[name][metric] = varexp_metrics[name].get(metric, 0) + batch_varexp_metrics[name][metric]
    
    for name in ce_metrics.keys():
        for metric in ce_metrics[name].keys():
            ce_metrics[name][metric] = ce_metrics[name][metric] / n_batches
        for metric in varexp_metrics[name].keys():
            varexp_metrics[name][metric] = varexp_metrics[name][metric] / n_batches
        
    return varexp_metrics, ce_metrics

varexp_metrics, ce_metrics = run_evaluation(trainer, buffer, n_batches=2, ce_batch_size=32)

print(f"Clean loss = {ce_metrics['mlp_0']['loss_original']:.3f}")
print()

for name in [k for k in ce_metrics.keys() if 'mlp' in k]:
    print(f"{name}   {ce_metrics[name]['loss_reconstructed'] - ce_metrics[name]['loss_original']:.3f}   {varexp_metrics[name]['frac_variance_explained']*100:.0f}%")

print()

for name in [k for k in ce_metrics.keys() if 'attn' in k]:
    print(f"{name}   {ce_metrics[name]['loss_reconstructed'] - ce_metrics[name]['loss_original']:.3f}   {varexp_metrics[name]['frac_variance_explained']*100:.0f}%")